### All Imports

In [ ]:
import siamese_face as face
import siamese_iris as iris

### Face Section

In [ ]:
# Define constants
data_dir = "Dataset/VISA_Face/VISA_Face"


# Load and preprocess dataset
class_dirs = face.os.listdir(data_dir)
class_images = {class_dir: face.find_images(face.os.path.join(
    data_dir, class_dir)) for class_dir in class_dirs}

pairs = []
labels = []

for class_dir, images in class_images.items():
    for i in range(len(images)):
        for j in range(i + 1, len(images)):
            pairs.append((images[i], images[j]))
            labels.append(1 if class_dir == class_images else 0)

# Split dataset into train and test sets
pairs_train, pairs_test, labels_train, labels_test = face.train_test_split(
    pairs, labels, test_size=0.2, random_state=42)

# Create Siamese network branches
base_network = face.create_siamese_network(
    face.input_shape, face.embedding_dim)
input_a = face.layers.Input(shape=face.input_shape)
input_b = face.layers.Input(shape=face.input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Calculate L1 distance between the embeddings
distance = face.tf.abs(processed_a - processed_b)

# Output layer
output = face.layers.Dense(1, activation='sigmoid')(distance)

# Create Siamese network model
model = face.models.Model([input_a, input_b], output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    face.generate_batch(pairs_train, labels_train, face.batch_size),
    steps_per_epoch=len(labels_train) // face.batch_size,
    epochs=face.epochs,
    validation_data=face.generate_batch(
        pairs_test, labels_test, face.batch_size),
    validation_steps=len(labels_test) // face.batch_size
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    face.generate_batch(pairs_test, labels_test, face.batch_size),
    steps=len(labels_test) // face.batch_size
)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

### Iris Section

In [ ]:
# Define constants
data_dir = "Dataset/VISA_Iris/VISA_Iris"


# Load and preprocess dataset
class_dirs = os.listdir(data_dir)
pairs = []
labels = []

for class_dir in class_dirs:
    left_images = iris.find_images(os.path.join(data_dir, class_dir, "/L"))
    right_images = iris.find_images(os.path.join(data_dir, class_dir, "/R"))
    for left_image in left_images:
        for right_image in right_images:
            pairs.append((left_image, right_image))
            labels.append(1 if class_dir == left_image.split("/")[-2] else 0)

# Split dataset into train and test sets
pairs_train, pairs_test, labels_train, labels_test = iris.train_test_split(
    pairs, labels, test_size=0.2, random_state=42)

# Create Siamese network branches
base_network = iris.create_siamese_network(
    iris.input_shape, iris.embedding_dim)
input_a = iris.layers.Input(shape=iris.input_shape)
input_b = iris.layers.Input(shape=iris.input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Calculate L1 distance between the embeddings
distance = iris.tf.abs(processed_a - processed_b)

# Output layer
output = iris.layers.Dense(1, activation='sigmoid')(distance)

# Create Siamese network model
model = iris.models.Model([input_a, input_b], output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])


# Train the model
history = model.fit(
    iris.generate_batch(pairs_train, labels_train, iris.batch_size),
    steps_per_epoch=len(labels_train) // iris.batch_size,
    epochs=iris.epochs,
    validation_data=iris.generate_batch(
        pairs_test, labels_test, iris.batch_size),
    validation_steps=len(labels_test) // iris.batch_size
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    iris.generate_batch(pairs_test, labels_test, iris.batch_size),
    steps=len(labels_test) // iris.batch_size
)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")